In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import tqdm
import src.util as util

from sklearn.model_selection import train_test_split

# 1. Load config data

In [2]:
config_data = util.load_config()

# 2. Load raw data

In [3]:
def read_raw_data(config: dict) -> pd.DataFrame:
    raw_data_path = config['raw_dataset_path']
    selected_columns = config['int64_columns'] + config['object_columns']
    df = pd.read_csv(raw_data_path)[selected_columns]
    return df

In [4]:
raw_dataset = read_raw_data(config_data)
raw_dataset.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   JobLevel                 1470 non-null   int64 
 1   Age                      1470 non-null   int64 
 2   DistanceFromHome         1470 non-null   int64 
 3   YearsAtCompany           1470 non-null   int64 
 4   YearsInCurrentRole       1470 non-null   int64 
 5   YearsSinceLastPromotion  1470 non-null   int64 
 6   YearsWithCurrManager     1470 non-null   int64 
 7   MonthlyIncome            1470 non-null   int64 
 8   MonthlyRate              1470 non-null   int64 
 9   EnvironmentSatisfaction  1470 non-null   int64 
 10  JobSatisfaction          1470 non-null   int64 
 11  WorkLifeBalance          1470 non-null   int64 
 12  PerformanceRating        1470 non-null   int64 
 13  Attrition                1470 non-null   object
 14  Department               1470 non-null  

In [5]:
# Save raw dataset to file
util.pickle_dump(raw_dataset, config_data["raw_dataset_collected_path"])

# 3. Data Definition

To simplify the project, we have selected several attributes to be included:

Target:
1. Attrition: This is a binary variable indicating whether the employee has left the company or not. This is the primary target variable for most analyses using this dataset.


Employee’s basic information:
1. Department: This categorical variable represents the department in which an employee works, such as 'Sales', 'Research & Development', or 'Human Resources'.
2. JobRole: This categorical variable indicates the role of the employee within the company.
3. JobLevel: This is an ordinal variable that represents the level of job held by the employee, where a higher number indicates a more senior role.
4. Gender: This categorical variable indicates the gender of the employee.
5. Age: This is the age of the employee, measured in years.


Employee’s work experience and income:
1. YearsAtCompany: This is the number of years an employee has worked at the current company.
2. YearsInCurrentRole: This is the number of years an employee has been in their current role within the company.
3. YearsSinceLastPromotion: This is the number of years since the employee's last promotion.
4. YearsWithCurrManager: This is the number of years an employee has been with their current manager.
5. MonthlyIncome: This is the monthly income of the employee.


Employee’s satisfaction and performance rating:
1. EnvironmentSatisfaction: This ordinal variable reflects the employee's satisfaction with the work environment, where 1 'Low', 2 'Medium', 3 'High', and 4 'Very High'.
2. JobSatisfaction: This ordinal variable reflects the employee's satisfaction with their job, where 1 'Low', 2 'Medium', 3 'High', and 4 'Very High'.
3. WorkLifeBalance: This ordinal variable reflects the employee's work-life balance, where 1 'Bad', 2 'Good', 3 'Better', and 4 'Best'.
4. PerformanceRating: This ordinal variable reflects the employee's performance rating, where 1 'Low', 2 'Good', 3 'Excellent', and 4 'Outstanding'.


# 4. Data Validation

In [6]:
# Check data dimension
print(f'Rows : {raw_dataset.shape[0]}')
print(f'Cols : {raw_dataset.shape[1]}')

# Check data type each variable
print('\nColumns types:')
display(raw_dataset.info())

Rows : 1470
Cols : 18

Columns types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   JobLevel                 1470 non-null   int64 
 1   Age                      1470 non-null   int64 
 2   DistanceFromHome         1470 non-null   int64 
 3   YearsAtCompany           1470 non-null   int64 
 4   YearsInCurrentRole       1470 non-null   int64 
 5   YearsSinceLastPromotion  1470 non-null   int64 
 6   YearsWithCurrManager     1470 non-null   int64 
 7   MonthlyIncome            1470 non-null   int64 
 8   MonthlyRate              1470 non-null   int64 
 9   EnvironmentSatisfaction  1470 non-null   int64 
 10  JobSatisfaction          1470 non-null   int64 
 11  WorkLifeBalance          1470 non-null   int64 
 12  PerformanceRating        1470 non-null   int64 
 13  Attrition                1470 non-null   object
 14  De

None

In [7]:
# Check the range of data for each variable
raw_dataset.describe()

,JobLevel,Age,DistanceFromHome,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,MonthlyIncome,MonthlyRate,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,PerformanceRating
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,2.063946,36.923810,9.192517,7.008163,4.229252,2.187755,4.123129,6502.931293,14313.103401,2.721769,2.728571,2.761224,3.153741
std,1.106940,9.135373,8.106864,6.126525,3.623137,3.222430,3.568136,4707.956783,7117.786044,1.093082,1.102846,0.706476,0.360824
min,1.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1009.000000,2094.000000,1.000000,1.000000,1.000000,3.000000
25%,1.000000,30.000000,2.000000,3.000000,2.000000,0.000000,2.000000,2911.000000,8047.000000,2.000000,2.000000,2.000000,3.000000
50%,2.000000,36.000000,7.000000,5.000000,3.000000,1.000000,3.000000,4919.000000,14235.500000,3.000000,3.000000,3.000000,3.000000
75%,3.000000,43.000000,14.000000,9.000000,7.000000,3.000000,7.000000,8379.000000,20461.500000,4.000000,4.000000,3.000000,3.000000
max,5.000000,60.000000,29.000000,40.000000,18.000000,15.000000,17.000000,19999.000000,26999.000000,4.000000,4.000000,4.000000,4.000000


In [8]:
object_columns = [
    'Attrition',
    'Department', 
    'JobRole', 
    'Gender',
    'OverTime']

int64_columns = [
    'JobLevel', 
    'Age', 
    'DistanceFromHome',
    'YearsAtCompany', 
    'YearsInCurrentRole', 
    'YearsSinceLastPromotion', 
    'YearsWithCurrManager', 
    'MonthlyIncome', 
    'MonthlyRate',
    'EnvironmentSatisfaction', 
    'JobSatisfaction', 
    'WorkLifeBalance',
    'PerformanceRating']

columns_config = {
    'object_columns' : object_columns,
    'int64_columns' : int64_columns
}

print(yaml.dump(columns_config))

int64_columns:
- JobLevel
- Age
- DistanceFromHome
- YearsAtCompany
- YearsInCurrentRole
- YearsSinceLastPromotion
- YearsWithCurrManager
- MonthlyIncome
- MonthlyRate
- EnvironmentSatisfaction
- JobSatisfaction
- WorkLifeBalance
- PerformanceRating
object_columns:
- Attrition
- Department
- JobRole
- Gender
- OverTime



In [9]:
object_column_range = {}

for col in object_columns:
    object_column_range[f'{col}'] = list(raw_dataset[col].unique())

print(yaml.dump(object_column_range))

Attrition:
- 'Yes'
- 'No'
Department:
- Sales
- Research & Development
- Human Resources
Gender:
- Female
- Male
JobRole:
- Sales Executive
- Research Scientist
- Laboratory Technician
- Manufacturing Director
- Healthcare Representative
- Manager
- Sales Representative
- Research Director
- Human Resources
OverTime:
- 'Yes'
- 'No'



In [10]:
int64_column_range = {}

for col in int64_columns:
    int64_column_range[f'{col}'] = {
        'min' : min(raw_dataset[col]), 
        'max' : max(raw_dataset[col])
    }

print(yaml.dump(int64_column_range))

Age:
  max: 60
  min: 18
DistanceFromHome:
  max: 29
  min: 1
EnvironmentSatisfaction:
  max: 4
  min: 1
JobLevel:
  max: 5
  min: 1
JobSatisfaction:
  max: 4
  min: 1
MonthlyIncome:
  max: 19999
  min: 1009
MonthlyRate:
  max: 26999
  min: 2094
PerformanceRating:
  max: 4
  min: 3
WorkLifeBalance:
  max: 4
  min: 1
YearsAtCompany:
  max: 40
  min: 0
YearsInCurrentRole:
  max: 18
  min: 0
YearsSinceLastPromotion:
  max: 15
  min: 0
YearsWithCurrManager:
  max: 17
  min: 0



# Data Defense

In [11]:
def check_data(input_data, params):
    # Data type checking
    assert input_data.select_dtypes("object").columns.to_list() == params["object_columns"], "Error occurs in object columns"
    assert input_data.select_dtypes("int64").columns.to_list() == params["int64_columns"], "Error occurs in object columns"

    # Range data checking for obj columns
    for col in config_data["object_columns"]:
        assert set(input_data[f"{col}"]).issubset(set(params[f"{col}_range"])), f"Error occurs in {col} column range"

    # Range data checking for int64 columns
    for col in config_data["int64_columns"]:
        assert input_data[f"{col}"].between(params[f"{col}_range"]["min"], params[f"{col}_range"]["max"]).sum() == len(input_data), f"Error occurs in {col} column range"

In [12]:
check_data(raw_dataset, config_data)

# Data Splitting

In [13]:
x = raw_dataset[config_data["int64_columns"]+config_data["object_columns"]].drop(columns=['Attrition']).copy()
y = raw_dataset['Attrition'].copy()

print(x.info())

print(y.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   JobLevel                 1470 non-null   int64 
 1   Age                      1470 non-null   int64 
 2   DistanceFromHome         1470 non-null   int64 
 3   YearsAtCompany           1470 non-null   int64 
 4   YearsInCurrentRole       1470 non-null   int64 
 5   YearsSinceLastPromotion  1470 non-null   int64 
 6   YearsWithCurrManager     1470 non-null   int64 
 7   MonthlyIncome            1470 non-null   int64 
 8   MonthlyRate              1470 non-null   int64 
 9   EnvironmentSatisfaction  1470 non-null   int64 
 10  JobSatisfaction          1470 non-null   int64 
 11  WorkLifeBalance          1470 non-null   int64 
 12  PerformanceRating        1470 non-null   int64 
 13  Department               1470 non-null   object
 14  JobRole                  1470 non-null  

In [14]:
# First split, splitting train and test set with ratio 0.7:0.3 and do stratify splitting
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42, stratify = y)

# Second split, splitting test and valid set with ratio 0.5:0.5 and do stratify splitting
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [15]:
print(f"""
Train size:
{y_train.value_counts()}

Test size:
{y_test.value_counts()}

Validation size:
{y_valid.value_counts()}
""")


Train size:
No     863
Yes    166
Name: Attrition, dtype: int64

Test size:
No     185
Yes     36
Name: Attrition, dtype: int64

Validation size:
No     185
Yes     35
Name: Attrition, dtype: int64



In [16]:
util.pickle_dump(x_train, config_data["train_set_path"]["x"])
util.pickle_dump(y_train, config_data["train_set_path"]["y"])

util.pickle_dump(x_valid, config_data["valid_set_path"]["x"])
util.pickle_dump(y_valid, config_data["valid_set_path"]["y"])

util.pickle_dump(x_test, config_data["test_set_path"]["x"])
util.pickle_dump(y_test, config_data["test_set_path"]["y"])